#<b>Automatic License Plate Recognition and Character Extraction using YOLOv3 Object Detection Algorithm and Tesseract Optical Character Recognition Engine</b><br><br>

<u><b>YOLOv3 Primary Dataset Name</b></u>: Microsoft Common Objects in Context (MS-COCO)<br>
<u><b>YOLOv3 Primary Dataset Link</b></u>: https://cocodataset.org/#download<br>
<u><b>YOLOv3 License Plate Dataset Name</b></u>: Tunisian License Plates Detection Dataset<br><br>


<u><b>YOLOv3 for License Plate Detection Weights File Link</b></u>: https://www.kaggle.com/achrafkhazri/yolo-weights-for-licence-plate-detector?select=lapi.weights<br>
<u><b>YOLOv3 for License Plate Detection Configuration File Link</b></u>: https://www.kaggle.com/achrafkhazri/yolo-weights-for-licence-plate-detector?select=darknet-yolov3.cfg<br>
<u><b>YOLOv3 for License Plate Detection Class Lables File Link</b></u>: https://www.kaggle.com/achrafkhazri/yolo-weights-for-licence-plate-detector?select=classes.names<br><br>


##<b>Exploration of Enviroment, Installation of Dependencies, and Mounting of Google Drive</b>

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!cat /proc/meminfo

MemTotal:       13305320 kB
MemFree:         9013024 kB
MemAvailable:   12356592 kB
Buffers:          101328 kB
Cached:          3317288 kB
SwapCached:            0 kB
Active:          1446868 kB
Inactive:        2522368 kB
Active(anon):     474676 kB
Inactive(anon):      432 kB
Active(file):     972192 kB
Inactive(file):  2521936 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1416 kB
Writeback:             0 kB
AnonPages:        550644 kB
Mapped:           261600 kB
Shmem:              1208 kB
KReclaimable:     175488 kB
Slab:             227892 kB
SReclaimable:     175488 kB
SUnreclaim:        52404 kB
KernelStack:        5360 kB
PageTables:         6740 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6652660 kB
Committed_AS:    3704948 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       47716 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5f9edcef-3d40-849b-f61e-f6396c48002a)


In [ ]:
!nvidia-smi

Sun May 16 09:49:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##<b>Import Packages</b>

In [ ]:
import os
import re
import cv2
import time
import math
import shutil
import random
import imutils
import pytesseract
import numpy as np
from PIL import Image
import IPython.core.display as ipd

##<b>Helper Functions</b>

In [ ]:
def convert(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

In [ ]:
def preprocess_plates(image, curFrame, x, y, w, h):

  if not os.path.exists("/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Vehicle"):
    os.makedirs("/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Vehicle")       
  if not os.path.exists("/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Plates"):
    os.makedirs("/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Plates")
  if not os.path.exists("/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Preprocessed"):
    os.makedirs("/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Preprocessed")
  

  name = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Vehicle/frame"+str(curFrame)+".jpg"
  name2 = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Plates/plateframe"+str(curFrame)+".jpg"
  name3 = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Preprocessed/licenseplate"+str(curFrame)+".jpg"

  cv2.imwrite(name,image)
  image=Image.open(name)
  crpimg=image.crop((x, y, x+w, y+h))
  crpimg.save(name2)

  image = cv2.imread(name2)
  final_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
  final_image = cv2.morphologyEx(final_image, cv2.MORPH_BLACKHAT, rectKern)
  clahe = cv2.createCLAHE(clipLimit = 7, tileGridSize = (1,200))
  final_image = clahe.apply(final_image)
  final_image = cv2.threshold(final_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

  cv2.imwrite(name3,final_image)

In [ ]:
def convert_to_mp4(AVI_VIDEO_PATH):

  MP4_DIRECTORY = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Result_Videos/MP4_Format/"

  result_video_name = AVI_VIDEO_PATH.split("/")[-1]
  result_video_name = result_video_name.split(".")[-2]
  MP4_VIDEO_PATH = MP4_DIRECTORY + result_video_name

  os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = AVI_VIDEO_PATH, output = MP4_VIDEO_PATH))
  MP4_VIDEO_PATH = MP4_VIDEO_PATH + str(".mp4")
  
  return MP4_VIDEO_PATH

In [ ]:
final_result = []
result_videos = []

def save_results(MP4_VIDEO_PATH, number_list):
  
  result = set(number_list)
  final_result_slice = []

  for x in result:
    if not x == 'THERE ARE NO LICENSE PLATES IN THE GIVEN VIDEO':
      clean_ans = re.findall("^TN[0-9]{1,2}[A-Z]{1,2}[0-9]{4}", x)
      clean_ans_str = ''.join([str(elem) for elem in clean_ans])
      if clean_ans:
        final_result_slice.append(clean_ans_str)
    else:
      final_result_slice.append(x)
  
  final_result.append(final_result_slice)
  result_videos.append(MP4_VIDEO_PATH)

##<b>YOLOv3 - License Plate Detector</b>

In [ ]:
def detect_and_store_license_plate(VIDEO_PATH):


  weightsPath = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/LPyolov3/LicensePlateYOLOv3.weights"
  configPath = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/LPyolov3/LicensePlateYOLOv3.cfg"
  labelsPath = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/LPyolov3/classes.names"
  LABELS = open(labelsPath).read().strip().split("\n")

  AVI_DIRECTORY = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Result_Videos/AVI_Format/"
  result_video_name = VIDEO_PATH.split("/")[-1]
  result_video_name = result_video_name.split(".")[-2]
  AVI_VIDEO_PATH = AVI_DIRECTORY + result_video_name + str("-Result.avi")

  net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
  cap = cv2.VideoCapture(VIDEO_PATH)

  curFrame=0
  writer = None
  (W, H) = (None, None)

  try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
      else cv2.CAP_PROP_FRAME_COUNT
    total = int(cap.get(prop))
    print("[INFO] {} Total Frames in Video".format(total))

  except:
    print("[INFO] Could Not Determine # of Frames in Video")
    print("[INFO] No Approximate Completion Time Can Be Provided")
    total = -1

  while(cap.isOpened()):
      
      ret,image=cap.read()
      if not ret:
          break
      image=cv2.resize(image,(1280,720))
      (H, W) = image.shape[:2]
      ln = net.getLayerNames()
      ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
      blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
      net.setInput(blob)
      start = time.time()
      layerOutputs = net.forward(ln)
      end = time.time()
      print("Frame Prediction Time : {:.6f} seconds".format(end - start))
      boxes = []
      confidences = []
      classIDs = []
      for output in layerOutputs:
          for detection in output:
              scores = detection[5:]
              classID = np.argmax(scores)
              confidence = scores[classID]
              if confidence > 0.1:
                  box = detection[0:4] * np.array([W, H, W, H])
                  (centerX, centerY, width, height) = box.astype("int")
                  x = int(centerX - (width / 2))
                  y = int(centerY - (height / 2))
                  boxes.append([x, y, int(width), int(height)])
                  confidences.append(float(confidence))
                  classIDs.append(classID)
                  
      idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
      ind = []
      for i in range(0,len(classIDs)):
          ind.append(i)

      if len(idxs) > 0:
          for i in idxs.flatten():

                  (x, y) = (boxes[i][0], boxes[i][1])
                  (w, h) = (boxes[i][2], boxes[i][3])
                  preprocess_plates(image, curFrame, x, y, w, h)
                  curFrame+=1
                  color = (0,255,0)
                  cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                  text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
                  cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)   
      
      if writer is None:
          fourcc = cv2.VideoWriter_fourcc(*"MJPG")
          writer = cv2.VideoWriter(AVI_VIDEO_PATH, fourcc, 24, (image.shape[1], image.shape[0]), True)
          if total > 0:
              elap = (end - start)
              print("[INFO] Single Frame Took {:.4f} Seconds".format(elap))
              print("[INFO] Estimated Total Time To Finish(HH:MM:SS): ",convert(math.ceil(elap * total)))
              
      writer.write(image)

  print("[INFO] Cleaning Up...")
  print("")
  print("")
  writer.release()
  cap.release()
  cv2.destroyAllWindows()
  return AVI_VIDEO_PATH

##<b>Tessaract OCR - License Number Extractor</b>

In [ ]:
def license_number_extractor(PREPROCESSED_IMAGE_PATH, number_list):

  text = pytesseract.image_to_string(Image.open(PREPROCESSED_IMAGE_PATH))
  text = text.upper()
  cleaned_list = re.findall("\w", text)
  temp_cleaned_str = ''.join([str(elem) for elem in cleaned_list])
  number_list.append(temp_cleaned_str)

  return number_list

##<b>Digital Assignment Solution Driver Function</b>

In [ ]:
def OSVKM_digital_assigment_solution(PATH_TO_VIDEO):

  number_list = []
  STATUS0 = 'THERE ARE NO LICENSE PLATES IN THE GIVEN VIDEO'
  AVI_VIDEO_PATH = detect_and_store_license_plate(PATH_TO_VIDEO)
  MP4_VIDEO_PATH = convert_to_mp4(AVI_VIDEO_PATH)
  DIRECTORY = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Preprocessed/"

  if not os.path.exists(DIRECTORY):
    number_list.append(STATUS0)

  else:
    for license_plate in os.listdir(DIRECTORY):
      if license_plate.endswith('.jpg'):
        PREPROCESSED_IMAGE_PATH = DIRECTORY + str(license_plate)
        number_list = license_number_extractor(PREPROCESSED_IMAGE_PATH, number_list)
  
  save_results(MP4_VIDEO_PATH, number_list)

  if os.path.exists(DIRECTORY):
    shutil.rmtree(DIRECTORY)

##<b>Evaluation on Videos in Test Video Folder</b>

In [ ]:
TEST_VIDEOS_PATH = "/content/drive/MyDrive/OSVKM_DA2_YOLOv3_License_Plate_Detection/Test_Videos/"

for video in os.listdir(TEST_VIDEOS_PATH):
  if video.endswith(".mp4"):
    PATH_TO_VIDEO = TEST_VIDEOS_PATH + str(video)
    OSVKM_digital_assigment_solution(PATH_TO_VIDEO)
    
print("")
print("---------- RESULTS GENERATED SUCESSFULLY ---------- ")

[INFO] 62 Total Frames in Video
Frame Prediction Time : 2.267058 seconds
[INFO] Single Frame Took 2.2671 Seconds
[INFO] Estimated Total Time To Finish(HH:MM:SS):  00:02:21
Frame Prediction Time : 1.772276 seconds
Frame Prediction Time : 1.755166 seconds
Frame Prediction Time : 1.764203 seconds
Frame Prediction Time : 1.797634 seconds
Frame Prediction Time : 1.747918 seconds
Frame Prediction Time : 1.779202 seconds
Frame Prediction Time : 1.755338 seconds
Frame Prediction Time : 1.771159 seconds
Frame Prediction Time : 1.759552 seconds
Frame Prediction Time : 1.779838 seconds
Frame Prediction Time : 1.733917 seconds
Frame Prediction Time : 1.724463 seconds
Frame Prediction Time : 1.741395 seconds
Frame Prediction Time : 1.714799 seconds
Frame Prediction Time : 1.711004 seconds
Frame Prediction Time : 1.770009 seconds
Frame Prediction Time : 1.735803 seconds
Frame Prediction Time : 1.730338 seconds
Frame Prediction Time : 1.754730 seconds
Frame Prediction Time : 1.733773 seconds
Frame Pr

##<b>Display Results</b>

In [ ]:
for i in range(0,len(result_videos)):
  print("RESULTS FROM VIDEO:",i+1)
  print("")
  print("LICENSE PLATE NUMBERS DETECTED FROM VIDEO:",i+1)
  print("")
  print(final_result[i])
  print("")
  print("VIDEO (WITH DETECTIONS):",i+1)
  print("")
  ipd.display(ipd.Video(result_videos[i],embed=True))
  print("")
  print("---------------------------------------------------------------------------------------------------------------------------------------------------------")
  print("")
  print("")

RESULTS FROM VIDEO: 1

LICENSE PLATE NUMBERS DETECTED FROM VIDEO: 1

['TN12D8038', 'TN12OO8037', 'TN12D8087', 'TN42D8037', 'TN12D8037', 'TN12D8037', 'TN12D0803']

VIDEO (WITH DETECTIONS): 1




---------------------------------------------------------------------------------------------------------------------------------------------------------


RESULTS FROM VIDEO: 2

LICENSE PLATE NUMBERS DETECTED FROM VIDEO: 2

['THERE ARE NO LICENSE PLATES IN THE GIVEN VIDEO']

VIDEO (WITH DETECTIONS): 2




---------------------------------------------------------------------------------------------------------------------------------------------------------


RESULTS FROM VIDEO: 3

LICENSE PLATE NUMBERS DETECTED FROM VIDEO: 3

['TN07CU0558']

VIDEO (WITH DETECTIONS): 3




---------------------------------------------------------------------------------------------------------------------------------------------------------




#<b><u>OBSERVATION REPORT</u>:</b><BR>

<p align="justify"> A detailed obaservation report is attached with this submission folder, in Portable Document Format (PDF). Along with that, I have also attached the Google Drive link for the report below,<br><br>

<b><i><u>Observation Report Link</u>:</i></b> https://drive.google.com/file/d/1eJHYVcWnTzPkfL2WAdvcT5rd38Q4hyH4/view?usp=sharing<br><br>

<p align="justify"><b><u>Note</u>:</b> The Result Video folder is also included in thate submission folder. The result videos are available in both MP4 and AVI formats. In the test videos folder, two videos contain License Plate and the other one does not. This is to ensure that the algorithm works for either of the cases.</p>